In [1]:
import re
# download data from from Google Drive
!gdown 1uml6lt5diPDjBnJYVECBrafVJEISRU2-

Downloading...
From: https://drive.google.com/uc?id=1uml6lt5diPDjBnJYVECBrafVJEISRU2-
To: /content/Manchester-morph+changes_utterancesremoved.cha
100% 25.3M/25.3M [00:00<00:00, 56.4MB/s]


In [3]:
# read the file in to a list of tokens
tokens=[]
filename="Manchester-morph+changes_utterancesremoved.cha"
samplesize=100000
with open(filename) as file:
    while len(tokens) <= samplesize :
        for line in file:
            if re.match("^\*MOT",line):
                line=re.sub('\*[A-Z]+\:','', line)
                line = "{ " + line + " } "
                tokens.extend(line.split())

In [10]:
# Select target based on frequency
from collections import Counter
counts=Counter(tokens)
# Sort the tuples. The reverse argument instructs to put most frequent first rather than last (which is the default)
so=sorted(counts.items(), key=lambda item: item[1], reverse=True)
# Extract the list of tokens, by transposing the list of lists so that there is a list of tokens a list of counts and then just selecting the former
so=list(zip(*so))[0]
# Select the firs 3500 words in the list
vocab_size = 1500
#type_list = so
type_list=so[500:(500+vocab_size)]
vocab_size = len(type_list)
token_count = len(tokens)

In [23]:
from collections import defaultdict
import numpy as np
type_count = len(type_list)
targets = []
positions = []
contextwordcounts = []
d = defaultdict(lambda: [0] * vocab_size)
window_size = 2
#M = np.zeros((type_count, 4, type_count))

for i, word in enumerate(tokens):
      if type_list.count(word) > 0:
            # Find the index in the tokenized sentence vector for the beginning of the window (the current token minus window size or zero whichever is the lower)
            begin = max(i - window_size, 0)
            # Find the index in the tokenized sentence vector for the end of the window (the current token plus window size or the length of the sentence whichever is the lower)
            end  = min(i + window_size, token_count)
            # Extract the text from beginning of window to the end
            context = tokens[begin: end + 1]
            # Remove the target word from its own window
            context.remove(tokens[i])
            # Iterate over the window for this target word
            target_id = type_list.index(word)
            #print(word + " " + str(context))
            p1 = context[1] + "_X_" + context[2]
            p1 = re.sub("[^\{]+(\{.+)","\\1",p1)
            p1 = re.sub("(.+\})[^\}]+","\\1",p1)

            p1a = context[1] + "_X_"
            p1a = re.sub("[^\{]+(\{.+)","\\1",p1)
            p1a = re.sub("(.+\})[^\}]+","\\1",p1)

            p2 = "X_" + context[2] + "_" + context[3]
            p2 = re.sub("[^\{]+(\{.+)","\\1",p2)
            p2 = re.sub("(.+\})[^\}]+","\\1",p2)

            p2a = "X_" + context[2]
            p2a = re.sub("[^\{]+(\{.+)","\\1",p2a)
            p2a = re.sub("(.+\})[^\}]+","\\1",p2a)

            p3 = context[0] + "_" + context[1] + "_X"
            p3 = re.sub("[^\{]+(\{.+)","\\1",p3)
            p3 = re.sub("(.+\})[^\}]+","\\1",p3)

            p3a = context[1] + "_X"
            p3a = re.sub("[^\{]+(\{.+)","\\1",p3a)
            p3a = re.sub("(.+\})[^\}]+","\\1",p3a)

            # Find the ID and hence the list index for the current token
            #token_id = type_list.index(token)
            # Add 1 to the current context dimension for the current target word
            d[p1][target_id] += 1
            d[p2][target_id] += 1
            d[p3][target_id] += 1
            d[p1a][target_id] += 1
            d[p2a][target_id] += 1
            d[p3a][target_id] += 1

            #M[current_row][position][token_id] += 1



In [24]:
# Convert to a hierachical data frame
import pandas as pd
df=pd.DataFrame(d)
df.index = type_list

In [25]:
df

,{_X_Mummy,X_Mummy_?,{_X,X_Mummy,and_X_come,X_come_on,Nanny_and_X,X_come,and_X,well_X_them,...,Charlie_X,X_(i)s_not,pushing_X_not,{_pushing_X,can_X_pull,"X_pull_,",coupled_X_you,just_coupled_X,coupled_X,X_little_stuck
always,2,0,54,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
tunnel,0,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hands,0,0,24,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
trying,0,0,40,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
paper,0,0,20,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
doctors,0,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
heads,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
I>,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
stays,0,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
